In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf


data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 16]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [16]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 16, 32], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[32]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 32, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step 100, entropy los

较少卷积核的数量效果并不是特别好

接下来将第一层卷积层的卷积核数量调整为64，第二层神经元数量调整为32

In [2]:

data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 64]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [64]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 64, 32], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[32]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 32, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
step 100, entropy loss: 0.889768, l2_loss: 12864.052734, total loss: 1.790252
0.76
step 200, entropy loss: 0.603610, l2_loss: 12862.458984, total loss: 1.503983
0.83
step 300, entropy loss: 0.422788, l2_loss: 12860.855469, total loss: 1.323048
0.89
step 400, entropy loss: 0.284134, l2_loss: 12859.197266, total loss: 1.184278
0.91
step 500, entropy loss: 0.240793, l2_loss: 12857.453125, total loss: 1.140815
0.85
step 600, entropy loss: 0.190240, l2_loss: 12855.747070, total loss: 1.090143
0.94
step 700, entropy loss: 0.350080, l2_loss: 12854.035156, total loss: 1.249862
0.89
step 800, entropy loss: 0.276901, l2_loss: 12852.297852, total loss: 1.176561
0.94
step 900, entropy loss: 0.217364, l2_loss: 12850.58203

效果也不好，如果两层卷积核数量均为64如何？

In [3]:

data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 64]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [64]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 64, 64], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
step 100, entropy loss: 0.750156, l2_loss: 25897.203125, total loss: 2.562960
0.74
step 200, entropy loss: 0.450031, l2_loss: 25893.597656, total loss: 2.262583
0.85
step 300, entropy loss: 0.528245, l2_loss: 25890.000000, total loss: 2.340545
0.88
step 400, entropy loss: 0.380238, l2_loss: 25886.410156, total loss: 2.192287
0.89
step 500, entropy loss: 0.308016, l2_loss: 25882.775391, total loss: 2.119810
0.89
step 600, entropy loss: 0.401105, l2_loss: 25879.142578, total loss: 2.212645
0.87
step 700, entropy loss: 0.298286, l2_loss: 25875.542969, total loss: 2.109574
0.95
step 800, entropy loss: 0.322712, l2_loss: 25871.923828, total loss: 2.133747
0.9
step 900, entropy loss: 0.265335, l2_loss: 25868.316406

结果相较于初始卷积核数量并没有太大的改进

再将两层卷积核军调整为32

In [5]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)

#构造一个reshape函数，重构输入图片数据大小
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

#第一个卷积层
with tf.name_scope('conv1'):
    shape = [6, 6, 1, 32]  
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

with tf.name_scope('pool1'):
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([6,6 , 32, 32], stddev=0.1),
                        
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[32]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                         padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)


with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')


with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 32, 1024], stddev=0.1),
                      
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                      collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)


for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

step 100, entropy loss: 1.184723, l2_loss: 38715.769531, total loss: 3.894827
0.68
step 200, entropy loss: 0.513702, l2_loss: 38710.675781, total loss: 3.223449
0.82
step 300, entropy loss: 0.519051, l2_loss: 38705.468750, total loss: 3.228434
0.88
step 400, entropy loss: 0.293234, l2_loss: 38700.218750, total loss: 3.002249
0.92
step 500, entropy loss: 0.257291, l2_loss: 38694.972656, total loss: 2.965939
0.92
step 600, entropy loss: 0.210056, l2_loss: 38689.667969, total loss: 2.918333
0.94
step 700, entropy loss: 0.276487, l2_loss: 38684.394531, total loss: 2.984395
0.96
step 800, entropy loss: 0.229309, l2_loss: 38679.035156, total loss: 2.936842
0.94
step 900, entropy loss: 0.210562, l2_loss: 38673.695312, total loss: 2.917721
0.95
step 1000, entropy loss: 0.322034, l2_loss: 38668.382812, total loss: 3.028821
0.92
0.9351
step 1100, entropy loss: 0.127375, l2_loss: 38663.039062, total loss: 2.833788
0.96
step 1200, entropy loss: 0.239356, l2_loss: 38657.730469, total loss: 2.945397

结果仍然一般，显然，64+32的卷积核数量性能已经还算不错了